In [1]:
from kerax.layers import Concatenate
from kerax.layers import Conv2D, Dense, Flatten, Input, BatchNormalization, Activation, Add
from kerax.models import Model
from kerax.losses import CategoricalCrossEntropy
import numpy as np
from copy import copy
from kerax.initializers import *
import jax.numpy as jnp

In [2]:
train_x = np.random.random((128, 28,28,1))
train_y = np.random.random((128, 10))

val_x = np.random.random((128, 28,28,1))
val_y = np.random.random((128, 10))

In [3]:
inputs = Input((28, 28, 1))
conv1 = Conv2D(64, 3)(inputs)
res = Conv2D(64, 3)(conv1)
act1 = Activation('relu')(conv1)
conv2 = Conv2D(128, 3)(act1)
act2 = Activation('relu')(conv2)
conv3 = Conv2D(128, 3, padding='same')(act2)
flatten = Flatten()(conv3)
dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(10)(dense1)
outputs = Activation('softmax')(dense2)
model = Model(input=inputs, output=outputs)

In [4]:
from kerax.engine.trackable import Tracker

In [7]:
res.get_output()

Exception: This layer does not have an output yet, call method should be called

In [57]:
conv = Conv2D(64, 3)

In [58]:
conv(train_x)

DeviceArray([[[[ 9.46676210e-02,  2.75130272e-02, -1.46796688e-01, ...,
                -6.21511564e-02, -1.37431830e-01,  7.81040266e-02],
               [ 6.41109645e-02,  5.36982808e-03, -1.39317866e-02, ...,
                -1.40456511e-02, -1.68211862e-01,  1.70689046e-01],
               [ 4.73266877e-02, -3.87512334e-02, -1.08279169e-01, ...,
                -5.67203984e-02, -9.33277532e-02,  1.89791843e-01],
               ...,
               [ 5.77841625e-02,  1.05049126e-01, -3.68282534e-02, ...,
                 8.76378790e-02, -1.86169237e-01,  1.24448352e-01],
               [ 1.32774442e-01, -6.07909523e-02, -4.26426753e-02, ...,
                -2.16232501e-02, -1.59375906e-01,  1.56002074e-01],
               [ 4.78455685e-02, -1.91173553e-02, -1.79391131e-01, ...,
                -2.79638693e-02, -1.21189088e-01,  6.35242909e-02]],

              [[ 1.10582821e-01,  5.32988273e-02, -1.84586748e-01, ...,
                 9.08285603e-02, -2.11067125e-01,  1.20592900e-01]

<function jax._src.nn.initializers.variance_scaling.<locals>.init(key, shape, dtype=<class 'jax._src.numpy.lax_numpy.float32'>)>